# Analogy Evaluation

## Load in the embeddings

In [ ]:

import pandas as pd
import numpy as np
import pickle
from itertools import islice
from sklearn import preprocessing  # Two samples, with 3 dimensions.
import analyze
import evaluate
import imp
import time
from sklearn.metrics import pairwise_distances

path = './data/'

embedding_names = ['bow2','bow5', 'deps']


simlexf = 'Simlex/SimLex-999.txt'

#
# embeddings: list of dicts holding the data for each embedding
# dict: {name - embedding name,
#       filename - embedding data filename
#       words - dict holding the words  and their vectors
#    
embeddings = []


for name in embedding_names:
    # words: dict, holds all words and their vectors for each embedding
    words = {}
    words_normalized = {}
    filename = name + '.words.bz2'
    embedding_df = pd.read_table(path + filename , sep=' ', header=None)
    for index, row in embedding_df.iterrows():
        words[row[0]] = row[1:]
    
    
    embedding_vectors = np.stack(list(words.values()))
    embedding_vocabulary = np.stack(list(words.keys()))
    embeddings.append({'name':name, 'filename': filename, 
                            'words': words, ' embedding_vectors':  embedding_vectors,
                       'embedding_vocabulary': embedding_vocabulary})
        



## ANALOGY TASK

In [ ]:





def MRR(X, Y, indices, exclude):
    # 
    # X: query vectors matrix (size: A x M)
    # Y: embedding vectors matrix (size: Nwords x Mfeatures)
    # indices: indices of the real (size: A)
    # exclude = list of indices to exclude for each analogy word
    # Compute distances between each codeword and each other codeword
    distance_matrix = scipy.spatial.distance.cdist(X, Y, metric='cosine')
    exclude = np.array(exclude)
    m = np.amax(distance_matrix)
    
    for i in [0,1,2]:
        #print(exclude[:,0].flatten())
        distance_matrix[np.arange(X.shape[0]), exclude[:,i].flatten()] =  m
    # Rank is the number of distances smaller than the correct distance, a
    # specified by the indices arg
    n_le = distance_matrix.T <= distance_matrix[np.arange(X.shape[0]), indices]
    n_lt = distance_matrix.T < distance_matrix[np.arange(X.shape[0]), indices]
    return (np.mean(1./n_le.sum(axis=0)),
           np.mean(1./(n_lt.sum(axis=0) + 1)))


MRR(all_bstars, embedding_vectors, answer_embedding_indexes, to_exclude)

In [4]:


analyze = imp.reload(analyze)

words = embeddings[0]['words']
embedding_vectors = np.stack(list(words.values()))
embedding_vocabulary = np.stack(list(words.keys()))

### Load in the analogy dataset

In [5]:
# Analogy Task

analogy_file = 'questions-words.txt'
analogy_words = pd.read_table(path + analogy_file , sep=' ', header=1)
analogy_words.head()



,Athens,Greece,Baghdad,Iraq
0,Athens,Greece,Bangkok,Thailand
1,Athens,Greece,Beijing,China
2,Athens,Greece,Berlin,Germany
3,Athens,Greece,Bern,Switzerland
4,Athens,Greece,Cairo,Egypt


In [55]:
start =  time.time()

all_bstars = []

for embedding in embeddings:
    words = embedding['words']
    embedding_vectors = np.stack(list(words.values()))
    embedding_vocabulary = np.stack(list(words.keys()))

    # warning here I only used the 9000 first words!
    # also breaks on first one
    indexes_found = []
    answers = []
    
    answer_embedding_indexes = []
    to_exclude = []
    all_bstars = []
    for index, row in analogy_words.iterrows():
            if row[0] in words.keys() and row[1] in words.keys() and row[2] in words.keys() and row[3] in words.keys():   
                #only do for a subset:
                indexes_found.append(index)
                a = words[row[0]]
                astar = words[row[1]]
                b = words[row[2]]
                vocab_keys = list(embedding_vocabulary)

                actual_bstar_index = vocab_keys.index(row[3])

                # Compute offset a* - a, add to b
                bstar = b.T + (astar.T - a.T)            
                cosine_sims = []
                cosine_sims_keys = []
                # Now that we computed  bstar find the closest vector to it
                # compute cosine similarity with all vectors
                # obtain the closest 

                #exclude = [row[0], row[1], row[2]]

              
                to_exclude.append([vocab_keys.index(row[0]),
                                  vocab_keys.index(row[1]),
                                  vocab_keys.index(row[2])])
                all_bstars.append(bstar)
                answer_embedding_indexes.append(actual_bstar_index)
    
    all_bstars_np = np.array(all_bstars)



    embeddingMRR = MRR(all_bstars_np, embedding_vectors, 
                       answer_embedding_indexes, to_exclude)
    
    print(embeddingMRR,len(indexes_found ))
    embedding.update({'MRR':embeddingMRR, 
                      'analogy_words_found': len(indexes_found)})
                      
end = time.time()
print(end-start, "seconds")
            

(0.70005419519385836, 0.70005419519385836) 9516
(0.69025550542683178, 0.69025550542683178) 9516
(0.67642604656615557, 0.67642604656615557) 9516
6475.316911935806 seconds


In [56]:
for embedding in embeddings:
    print(embedding['name'], embedding['MRR'])
    
    
    

bow2 (0.70005419519385836, 0.70005419519385836)
bow5 (0.69025550542683178, 0.69025550542683178)
deps (0.67642604656615557, 0.67642604656615557)


In [ ]:
import scipy


In [175]:
len(all_bstars)


1632

In [161]:
len(sorted_indices)
len(answer_embedding_indexes)


1632

In [162]:
sorted_indices_answers = sorted_indices[np.arange(len(sorted_indices)),
                                        answer_embedding_indexes]
                                        


In [163]:
sorted_indices_answers

array([  39,  402,  130, ...,  823, 1335,  673])

In [ ]:
np.shape(dists)



In [ ]:
# Get the answer 

In [ ]:
np.argsort()

In [142]:
# Find the indexes of each answer so as to obtain their rank
correct_words = 0 
answers_emb_index = []
for index, row in analogy_words.iterrows():
    if index in indexes_found:
        accurate_answer = row[3]
        answer_embedding_index = list(embedding_vocabulary).index(accurate_answer)
        answers_emb_index.append(answer_embedding_index)
        

        
    


In [143]:
answers_emb_index

[]

In [125]:
    print(accurate_answer, possible_answers, '\n')
        if accurate_answer in possible_answers[index]:
            correct_words += 1
            
            

In [ ]:
ind

In [126]:
import csv
import re
indexes_found = []
with open('answers_' + embedding['name'] + 'words_found.csv','r') as csvfile:
    reader = csv.reader(csvfile)
    for r in reader:
        indexes_found.append(int(str(r).strip('\[\]\'')))
        
possible_answers = []
with open('answers_' + embedding['name'] + '.csv','r') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        row_words =[]
        for w in row:
            wnew = re.sub('[\'\"\ ]', '', w)            
            row_words.append(wnew)
        possible_answers.append(row_words)
        
         
        
        

In [98]:
possible_answers[1][1]

'daughters'

In [127]:
indexes_found

[]

In [115]:
indexes_found

[8366,
 8367,
 8368,
 8369,
 8370,
 8371,
 8372,
 8373,
 8374,
 8375,
 8376,
 8377,
 8378,
 8379,
 8380,
 8381,
 8382,
 8383,
 8384,
 8385,
 8386,
 8387,
 8388,
 8389,
 8390,
 8391,
 8392,
 8393,
 8394,
 8395,
 8396,
 8397,
 8398,
 8399,
 8400,
 8401,
 8402,
 8403,
 8404,
 8405,
 8406,
 8407,
 8408,
 8409,
 8410,
 8411,
 8412,
 8413,
 8414,
 8415,
 8416,
 8417,
 8418,
 8419,
 8420,
 8421,
 8422,
 8423,
 8424,
 8425,
 8426,
 8427,
 8428,
 8429,
 8430,
 8431,
 8432,
 8433,
 8434,
 8435,
 8436,
 8437,
 8438,
 8439,
 8440,
 8441,
 8442,
 8443,
 8444,
 8445,
 8446,
 8447,
 8448,
 8449,
 8450,
 8451,
 8452,
 8453,
 8454,
 8455,
 8456,
 8457,
 8458,
 8459,
 8460,
 8461,
 8462,
 8463,
 8464,
 8465,
 8466,
 8467,
 8468,
 8469,
 8470,
 8471,
 8472,
 8473,
 8474,
 8475,
 8476,
 8477,
 8478,
 8479,
 8480,
 8481,
 8482,
 8483,
 8484,
 8485,
 8486,
 8487,
 8488,
 8489,
 8490,
 8491,
 8492,
 8493,
 8494,
 8495,
 8496,
 8497,
 8498,
 8499,
 8500,
 8501,
 8502,
 8503,
 8504,
 8505,
 8506,
 8507,
 8508,

In [118]:
 # ASK how many words for MRR
    

                

    


sister [['[sister', 'cousin', 'brother-in-law', 'sister-in-law]'], ['[sons', 'daughters', 'siblings', 'brothers-in-law]'], ['[mum', 'girlfriend', 'fiance', 'ex-boyfriend]'], ['[grandmother', 'stepfather', 'parents', 'husband]'], ['[great-grandfather', 'mother', 'aunt', 'great-grandmother]'], ['[annabelle', 'granny', 'maggie', 'sally]'], ['[daughter', 'niece', 'great-grandson', 'cousin]'], ['[bridegroom', 'wedding', 'hippodamia', 'bride-to-be]'], ['[her', 'marji', 'jae-min', 'ji-eun]'], ['[their', 'tokiko', 'step-sister', 'hers]'], ['[ex-husband', 'mother-in-law', 'sister-in-law', 'boyfriend]'], ['[monarch', 'king-emperor', 'princess', 'pmare]'], ['[stranger', 'person', 'divorcee', 'villager]'], ['[cousin', 'brother-in-law', 'son-in-law', 'granddaughter]'], ['[prostitute', 'woman', 'housewife', 'waitress]'], ['[hohenlohe-langenburg', 'vadhana', 'bourbon-parma', 'asfaw]'], ['[granddaughter', 'niece', 'grandson', 'cousin]'], ['[granddaughters', 'grandsons', 'half-sisters', 'stepchildren]'

IndexError: list index out of range

In [103]:
correct_words

0